<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->




# **P1 - RA ÍMPAR**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Enzo Moura, 21015350' #@param {type:"string"}

In [ ]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [ ]:
import pandas as pd

df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [ ]:
df_count_hosts = df.groupby('host_name').id.count().sort_values(ascending=False).head(3)
df_count_hosts

host_name
Sonder (NYC)    7
Mark            7
David           6
Name: id, dtype: int64

# Q2B. Preço nos bairros com mais imóveis

 Qual o preço médio dos imóveis, considerando somente os preços dos imóveis cujos bairros (`neighbourhood`) tenham mais de 20 imóveis sendo anunciados?  


In [ ]:
valid_neighbourhoods = df.groupby('neighbourhood').id.count() > 20
valid_neighbourhoods = valid_neighbourhoods[valid_neighbourhoods].index
preco_medio = df[df['neighbourhood'].isin(valid_neighbourhoods)]['price'].mean()

print(f'O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD {preco_medio:.2f}')

O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD 151.42


# Q3B.  Brooklyn *outliers*

Qual o percentual de imóveis de Brooklyn com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [ ]:
...
def identify_outliers(data):
  Q1 = data.quantile(0.25)
  Q3 = data.quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR
  outliers = data[(data < lower_bound) | (data > upper_bound)].index.tolist()
  return outliers

perc_Brooklyn = (len(identify_outliers(df[df['neighbourhood_group'] == 'Brooklyn']['price'])) / len(df[df['neighbourhood_group'] == 'Brooklyn'])) * 100
print( f'Percentuais de outliers de preços, Brooklyn: {perc_Brooklyn:.2f} %')


Percentuais de outliers de preços, Brooklyn: 5.76 %


# Q4B. Scores

Nesta tarefa você deve atribuir a cada imóvel um valor de *score* (`df['score']`) baseado nas seguintes regras:

1. Imóveis com `availability_365` > 90 recebem +10 pontos enquanto imóveis com `availability_365` < 5 recebem -5 pontos;
2. Imóveis com o número de reviews >= 10 recebem +5 pontos para cada 10 reviews (por ex. se número de reviews = 20, são atribuídos +10 pontos). Se número de reviews = 0 o imóvel recebe -5 pontos;
3. Imóveis com preço abaixo da média de seu `neighbourhood`	recebem +5 pontos.

Implemente empregando `apply()` ou um laço de programa, como preferir.

In [ ]:
...
def calculate_score(data):
    score = 0
    if data['availability_365'] > 90:
        score = 10
    elif data['availability_365'] < 5:
        score = -5
    if data['number_of_reviews'] >= 10:
        score += 5 * (data['number_of_reviews'] // 10)
    elif data['number_of_reviews'] == 0:
        score -= 5
    neighbourhood_mean = df[df['neighbourhood'] == data['neighbourhood']]['price'].mean()
    if data['price'] < neighbourhood_mean:
        score += 5
    return score

df['score'] = df.apply(calculate_score, axis=1)
print(f'A soma dos valores de score é: ', df['score'].sum())


A soma dos valores de score é:  10955


# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [ ]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     NaN
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     NaN
1524  20325804  fines     NaN
1525  25872986  fines     NaN
1526  19365218  fines   906.0
1527   4235843  fines     NaN

[1528 rows x 3 columns]

In [ ]:
df_taxes_nona = df_taxes.fillna(0)
df_taxes_nona = df_taxes_nona.merge(df, on='id', how='left')
df_taxes_aux = df_taxes_nona[df_taxes_nona['type']=='taxes']
df_fines_aux = df_taxes_nona[df_taxes_nona['type']=='fines']
taxes_mean = df_taxes_aux.groupby('neighbourhood_group').value.mean().astype(int)
fines_mean = df_fines_aux.groupby('neighbourhood_group').value.mean().astype(int)
print(f'A média de impostos por grupo de vizinhança é: {taxes_mean}')
print(f'A média de multas por grupo de vizinhança é: {fines_mean}')

A média de impostos por grupo de vizinhança é: neighbourhood_group
Bronx             851
Brooklyn          858
Manhattan         888
Queens            804
Staten Island    1157
Name: value, dtype: int64
A média de multas por grupo de vizinhança é: neighbourhood_group
Bronx            126
Brooklyn         314
Manhattan        314
Queens           322
Staten Island    581
Name: value, dtype: int64


# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [ ]:
df_aux = df.copy()[['neighbourhood_group','neighbourhood','price']]
df_aux = df_aux.sort_values(by=['neighbourhood_group','neighbourhood']).reset_index(drop=True)
df_aux = df_aux.set_index('neighbourhood_group')
df_lines = df_aux.groupby(['neighbourhood_group','neighbourhood']).agg({'price':'mean'}).reset_index()
df_lines
df_report = pd.DataFrame(columns=['page_id','neighbourhood_group','neighbourhood','price'])
df_report['neighbourhood_group'] = df_lines['neighbourhood_group']
df_report['neighbourhood'] = df_lines['neighbourhood']
df_report['price'] = df_lines['price']
df_report['page_id'] = (df_report.groupby('neighbourhood_group').cumcount() // 30) + 1
df_report






page_id neighbourhood_group      neighbourhood  price
0          1               Bronx            Belmont   29.0
1          1               Bronx  Claremont Village   67.0
2          1               Bronx          Concourse   40.0
3          1               Bronx           Edenwald   85.0
4          1               Bronx          Fieldston   97.0
..       ...                 ...                ...    ...
108        1       Staten Island      Randall Manor   99.0
109        1       Staten Island        South Beach   60.0
110        1       Staten Island         St. George  130.0
111        1       Staten Island          Todt Hill  135.0
112        1       Staten Island      Tompkinsville   59.0

[113 rows x 4 columns]

In [ ]:
sum_price_pages = df_report[df_report.page_id.isin([1,24,25,26,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

A soma dos preços das páginas 1, 24, 25 e 28 é: 15314


#### Aqui um *sample* do report.

In [ ]:
df_report.iloc[-10::,:]

page_id neighbourhood_group  neighbourhood       price
103        1              Queens      Sunnyside   85.857143
104        1              Queens     Whitestone  140.000000
105        1              Queens      Woodhaven  199.000000
106        2              Queens       Woodside   98.333333
107        1       Staten Island  Arden Heights   83.000000
108        1       Staten Island  Randall Manor   99.000000
109        1       Staten Island    South Beach   60.000000
110        1       Staten Island     St. George  130.000000
111        1       Staten Island      Todt Hill  135.000000
112        1       Staten Island  Tompkinsville   59.000000